Part 1: Scrap the wiki table and transfer it into a pandas data frame

In [46]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
canada = tables[0]
canada.columns = canada.iloc[0]
canada = canada.iloc[1:]
canada = canada[canada.Borough != 'Not assigned']
canada['Neighbourhood'].loc[canada['Neighbourhood'] == 'Not assigned'] =  canada.Borough

canada.reset_index(drop=True)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Part 2: add lat and long to the data set 

In [49]:
lat_long = pd.read_csv('/Users/tianyifan/Downloads/Geospatial_Coordinates.csv')

In [48]:
lat_long.shape

(103, 3)

In [50]:
canada_latlong = pd.merge(canada, lat_long, how='inner', on='Postcode')

In [54]:
canada_latlong.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
6,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [56]:
canada_latlong.shape

(210, 5)

Part 3: filter the data set to only boroughs containing "Tornoto", explore and cluster the data, create maps 

In [58]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - https://repo.continuum.io/pkgs/main/osx-64/anaconda-5.0.1-py36h6e48e2d_1.tar.bz2/osx-64::anaconda==5.0.1=py36h6e48e2d_1
- 
  - https://repo.continuum.io/pkgs/main/osx-64/bzip2-1.0.6-h92991f9_1.tar.bz2::bzip2-1.0.6-h92991f9_1, https://repo.continuum.io/pkgs/main/osx-64/intel-openmp-2018.0.0-h68bdfb3_7.tar.bz2::intel-openmp-2018.0.0-h68bdfb3_7, https://repo.continuum.io/pkgs/main/osx-64/jbig-2.1-h4d881f8_0.tar.bz2::jbig-2.1-h4d881f8_0, https://repo.continuum.io/pkgs/main/osx-64/libssh2-1.8.0-h1218725_2.tar.bz2::libssh2-1.8.0-h1218725_2
  - defaults::bzip2-1.0.6-h92991f9_1, https://repo.continuum.io/pkgs/main/osx-64/intel-openmp-2018.0.0-h68bdfb3_7.tar.bz2::intel-openmp-2018.0.0-h68bdfb3_7, https://repo.continuum.io/pkgs/main/osx-64/jbig-2.1-h4d881f8_0.tar.bz2::jbig-2.1-h4d881f8_0, https://repo.continuum.io/pkgs/main/osx-64/libssh2-1.8

pytest-remotedata-0. | 11 KB     | ##################################### | 100% 
pytest-openfiles-0.4 | 9 KB      | ##################################### | 100% 
liblapack-3.8.0      | 10 KB     | ##################################### | 100% 
pytest-doctestplus-0 | 16 KB     | ##################################### | 100% 
pytest-astropy-0.6.0 | 6 KB      | ##################################### | 100% 
soupsieve-1.9.4      | 57 KB     | ##################################### | 100% 
openssl-1.0.2t       | 3.1 MB    | ##################################### | 100% 
send2trash-1.5.0     | 12 KB     | ##################################### | 100% 
anaconda-custom      | 3 KB      | ##################################### | 100% 
_anaconda_depends-20 | 5 KB      | ##################################### | 100% 
pytest-arraydiff-0.3 | 11 KB     | ##################################### | 100% 
spyder-kernels-0.2.4 | 35 KB     | ##################################### | 100% 
ca-certificates-2019 | 143 K

vincent-0.4.4        | 28 KB     | ##################################### | 100% 
folium-0.5.0         | 45 KB     | ##################################### | 100% 
altair-3.2.0         | 750 KB    | ##################################### | 100% 
branca-0.3.1         | 25 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


In [60]:
canada_latlong_data = canada_latlong[canada_latlong['Borough'].str.contains('Toronto')].reset_index(drop=True)


In [61]:
canada_latlong_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [67]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto are 43.653963, -79.387207.


In [68]:
map_torontoneighborhood = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(canada_latlong_data['Latitude'], canada_latlong_data['Longitude'], canada_latlong_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontoneighborhood)  
    
map_torontoneighborhood

In [69]:
CLIENT_ID = 'ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I' # your Foursquare ID
CLIENT_SECRET = 'R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I
CLIENT_SECRET:R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ


In [70]:
canada_latlong_data.loc[0, 'Neighbourhood']

'Harbourfront'

In [71]:
neighborhood_latitude = canada_latlong_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = canada_latlong_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = canada_latlong_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [75]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd30f591a4b0a001b13674f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [77]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [79]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
neighbourhood_withTornoto_venues = getNearbyVenues(names=canada_latlong_data['Neighbourhood'],
                                   latitudes=canada_latlong_data['Latitude'],
                                   longitudes=canada_latlong_data['Longitude']
                                  )

Harbourfront
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and W

In [83]:
print(neighbourhood_withTornoto_venues.shape)
neighbourhood_withTornoto_venues.head()

(3237, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [85]:
neighbourhood_withTornoto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56
Brockton,23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,14,14,14,14,14,14
Cabbagetown,44,44,44,44,44,44
Central Bay Street,83,83,83,83,83,83
Chinatown,96,96,96,96,96,96


In [96]:
neighbourhood_withTornoto_venues_grouped =neighbourhood_withTornoto_venues.groupby('Neighborhood').mean().reset_index()

In [86]:
print('There are {} uniques categories.'.format(len(neighbourhood_withTornoto_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [97]:
# set number of clusters
kclusters = 5

tornoto_clustering = neighbourhood_withTornoto_venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tornoto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 3, 0, 0, 0, 2, 2], dtype=int32)

In [103]:
neighbourhood_withTornoto_venues_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
tornoto_merged = canada_latlong_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tornoto_merged = tornoto_merged.join(neighbourhood_withTornoto_venues_grouped.set_index('Neighborhood'), on='Neighbourhood')

tornoto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,43.654260,-79.360636,43.652805,-79.359606
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,43.657162,-79.378937,43.656118,-79.380503
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,43.657162,-79.378937,43.656118,-79.380503
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,43.656527,-79.373052,43.655173,-79.373581
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,43.676357,-79.293031,43.678861,-79.295440


In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tornoto_merged['Latitude'], tornoto_merged['Longitude'], tornoto_merged['Neighbourhood'], tornoto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters